# BioPython 

## By: Molly Davis 

## Email: mdavi258@uncc.edu

------------

## Step 1: Using SeqIO, read in and parse the file of apple primary transcripts (available on Canvas):
    - During your parsing, create a list of only transcripts of length 125 or less
    - Report the number of transcripts matching this criteria
    - Translate these sequences to protein, make sure to save them to their own list

In [1]:
from Bio import SeqIO 

file = "Mdomestica_491_v1.1.cds_primaryTranscriptOnly.fa"
parsed_data = list(SeqIO.parse(file, "fasta"))

Sequences = []
for rec in parsed_data:
    Sequences.append(rec)
print("Number of Available Sequences: \n" + str(len(Sequences))) 

short_sequences = []
for newrec in Sequences:
    if len(newrec.seq) <= 125:
        short_sequences.append(newrec)
print("Transcripts with a length of 125 or less: \n" + str(len(short_sequences)))

protein_translate = []
for prec in short_sequences:
    trans = prec.translate()
    protein_translate.append(str(trans.seq.strip("*")))
#print("\nTranslation of short sequences to proteins: \n\n" + str(protein_translate))

Number of Available Sequences: 
45116
Transcripts with a length of 125 or less: 
62


## Step 2: Align our small proteins using pairwise2
    - Align each protein to each other protein using the blosum62 scoring matrix, a gap opening penalty of -10, and a gap extension penalty of -0.5
    - If an alignment scores better than 40, save it.
    - Print the highest scoring pairwise alignment
    - Take care in how you set up your loops for the pairwise alignments. Each alignment only needs to be done once. If you've already tested the alignment of seq1 vs seq5, don't align seq5 vs seq1.
    

In [2]:
from Bio import pairwise2
from Bio.Align import substitution_matrices
blosum62 = substitution_matrices.load("BLOSUM62")
from Bio import Align

aligner= Align.PairwiseAligner() 
for i in range(0, len(protein_translate)):
    for j in range(i+1, len(protein_translate)): 
        score= aligner.score(protein_translate[i], protein_translate[j])
        if score >= 40:
            alignments = pairwise2.align.globalds(protein_translate[i], protein_translate[j], blosum62, -10, -0.5)
            print(alignments)
            print(pairwise2.format_alignment(*alignments[0]))    

[Alignment(seqA='MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL', seqB='MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL', score=211.0, start=0, end=40)]
MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL
||||||||||||||||||||||||||||||||||||||||
MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL
  Score=211



## Step 3: Running BLAST and reading results
     *Because we are doing a web BLAST, choose ONLY one sequence from our list of short proteins*.
    - Run BLAST with your sequence against the NR database
    - Parse the results. Report any HSPs with an E-value less than 0.05 and show the HSP alignments, including the name of the matching sequence. If no HSPs meet that criteria, report the highest scoring pair.
    - If for whatever reason the sequence you selected fails to return any results, try a new one 

In [3]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("SingleSequence1.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))

with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()
result_handle = open("my_blast.xml")
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)
blast_record = next(blast_records)

E_VALUE_THRESH = 0.05

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

****Alignment****
sequence: gi|1632217597|ref|XR_003776463.1| PREDICTED: Malus domestica uncharacterized LOC114827259 (LOC114827259), ncRNA
length: 391
e value: 2.28428e-54
ATGTACGGAGCAGGAGCTAAAGAAGATCTTCAACTATGTAGTGGTTGTCTTTGCACTTGTAGCGCTGAGGCACTG...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
ATGTACGGAGCAGGAGCTAAAGAAGATCTTCAACTATGTAGTGGTTGTCTTTGCACTTGTAGCGCTGAGGCACTG...
****Alignment****
sequence: gi|1632217597|ref|XR_003776463.1| PREDICTED: Malus domestica uncharacterized LOC114827259 (LOC114827259), ncRNA
length: 391
e value: 2.61097e-09
AGCGCTGAGGCACTGCAAATTCAAAGGAAGAGAAGAAGGGTTTATAACTTTCTATGTACGTAG...
||| |||||||||  |||||||||||  || ||||||||||||| ||||||| ||||||| ||...
AGCACTGAGGCACCACAAATTCAAAGAGAGCGAAGAAGGGTTTACAACTTTCGATGTACGGAG...
****Alignment****
sequence: gi|1732635877|ref|XM_030680407.1| PREDICTED: Rhodamnia argentea DIS3-like exonuclease 2 (LOC115744800), transcript variant X4, mRNA
length: 4313
e value: 0.0024454
CAAATTCAAAGGAAGAGAAGA--A